In [1]:

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
from scipy.linalg import expm
import pickle
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_heisenberg_qutip

boltzmann = lambda beta, w: np.min([1, np.exp(-beta * w)])
trace_dist = lambda rho1, rho2: 0.5 * np.trace(np.sqrt((rho1 - rho2).conj().T @ (rho1 - rho2)))

### State we should get

$$\rho' = \mathbb{I} + \delta \left[ \sum_\omega \gamma(\omega) \left(-\frac{1}{2} \{\sigma_x(\omega)^\dagger \sigma_x(\omega), \rho\} + \sigma_x(\omega) \rho \sigma_x(\omega)^\dagger\right) \right]+ \mathcal{O}(\delta^2)$$
Where we used the $\sigma_x$ jump on the 1st system qubit out of the [0, 1, 2]. Here it is still under inverstigation how large should $\delta$ be s.t we can really drop the higher orders, and also that we only have those terms there that should be dropped.

In [2]:
num_qubits = 3
num_energy_bits = 6
delta = 0.001 
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
T = 1
total_time = T #! * 2 * np.pi
num_trotter_steps = 10

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)

step_size = total_time / num_trotter_steps
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, 
                                      coeffs=hamiltonian.rescaled_coeffs, shift=(total_time*hamiltonian.shift))

# U = expm(1j * total_time * hamiltonian.qt.full())
# print(np.linalg.norm(U - trott_U_qt.full()))

#* Initial state
with open(f'data/initial_state_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    initial_state = pickle.load(f)

initial_dm = DensityMatrix(initial_state).data

Original spectrum:  [-3.7    -3.177  -2.5482 -1.7     1.625   2.4547  3.2521  3.7936]
Ideal spectrum:  [0.     0.0314 0.0692 0.1201 0.3198 0.3696 0.4175 0.45  ]
Nonrescaled coefficients:  [1.  0.8 0.9 1. ]
Rescaled coefficients:  [0.06005156 0.04804125 0.05404641 0.06005156]
eXX, eYY, eZZ for qubit (0, 1)
eXX, eYY, eZZ for qubit (1, 2)
eXX, eYY, eZZ for qubit (2, 0)


FileNotFoundError: [Errno 2] No such file or directory: 'data/initial_state_n3k6d0.001.pkl'

In [ ]:
N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
energy_labels = N_labels / N
energy_labels[0] = 0
print(energy_labels)

jump_op = qt.tensor([qt.qeye(2), qt.sigmax(), qt.qeye(2)]).full()

dm_we_should_get = np.eye(2**num_qubits, dtype=np.complex128)
#* Exact
# for w in energy_labels:
#     oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, hamiltonian=hamiltonian.qt.full())
#     dm_we_should_get += delta * boltzmann(w)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
#                                       -0.5 * initial_dm @ oft_op.conj().T @ oft_op
#                                       + oft_op @ initial_dm @ oft_op.conj().T)

#* Trotter
for w in energy_labels:
    oft_op = oft(jump_op, w, 2**num_energy_bits, sigma, trotter=trott_U_qt.full())
    dm_we_should_get += delta * boltzmann(beta, w)*(-0.5 * oft_op.conj().T @ oft_op @ initial_dm
                                                    -0.5 * initial_dm @ oft_op.conj().T @ oft_op
                                                    + oft_op @ initial_dm @ oft_op.conj().T)
    
dm_we_should_get /= np.trace(dm_we_should_get)


[ 0.        0.015625  0.03125   0.046875  0.0625    0.078125  0.09375
  0.109375  0.125     0.140625  0.15625   0.171875  0.1875    0.203125
  0.21875   0.234375  0.25      0.265625  0.28125   0.296875  0.3125
  0.328125  0.34375   0.359375  0.375     0.390625  0.40625   0.421875
  0.4375    0.453125  0.46875   0.484375 -0.5      -0.484375 -0.46875
 -0.453125 -0.4375   -0.421875 -0.40625  -0.390625 -0.375    -0.359375
 -0.34375  -0.328125 -0.3125   -0.296875 -0.28125  -0.265625 -0.25
 -0.234375 -0.21875  -0.203125 -0.1875   -0.171875 -0.15625  -0.140625
 -0.125    -0.109375 -0.09375  -0.078125 -0.0625   -0.046875 -0.03125
 -0.015625]


In [ ]:
the_time = 1
time_evolution_trott = lambda t: np.linalg.matrix_power(trott_U_qt, 1 * t)
time_evolution = lambda t: expm(1j * t * hamiltonian.qt.full())
np.linalg.norm(time_evolution(the_time) - time_evolution_trott(the_time))

0.0027514772762741737

In [ ]:
with open(f'data/state_before_measure_n{num_qubits}k{num_energy_bits}d{delta}.pkl', 'rb') as f:
    circ_state = pickle.load(f)

circ_sys_dm = partial_trace(circ_state, list(range(2 + num_energy_bits))).data
circ_sys_dm /= np.trace(circ_sys_dm)

In [ ]:
dist = np.linalg.norm(dm_we_should_get - circ_sys_dm)
print(f'Basic distance = {dist:.4f}')
tr_dist = trace_dist(dm_we_should_get, circ_sys_dm)
print(f'Trace distance = {tr_dist:.4f}')
fidelity = state_fidelity(DensityMatrix(dm_we_should_get), DensityMatrix(circ_sys_dm), validate=False)
print(f'Fidelity = {fidelity:.4f}')

tr_dist_qt = qt.tracedist(qt.Qobj(dm_we_should_get), qt.Qobj(circ_sys_dm))
print(f'Qutip trace distance = {tr_dist_qt:.4f}')
fid_qt = qt.fidelity(qt.Qobj(dm_we_should_get), qt.Qobj(circ_sys_dm))
print(f'Qutip fidelity = {fid_qt:.4f}')

Basic distance = 0.9350
Trace distance = 1.0776+0.0000j
Fidelity = 0.1349
Qutip trace distance = 0.8746
Qutip fidelity = 0.3672
